# Redes neuronales 2

En la sección anterior vimos cómo una neurona funcionaba como clasificador lineal. Pero, ¿qué podemos hacer si nuestro *dataset* no es separable linealmente? O, ¿qué ocurre si tenemos más de dos clases? La respuesta es que necesitamos una red neuronal.

Como su nombre indica, una red neuronal es un conjunto de neuronas interconectadas entre sí. ¿De qué forma? Antes de responder a esta pregunta, vamos a retroceder a nuestra primera definición de neurona, aquella en la que utilizábamos “mayor o igual” en lugar de la función sigmoide como función de activación. Lo vamos a hacer así, simplemente por una cuestión didáctica. Cuando entendamos bien el funcionamiento de la red, volveremos a usar la sigmoide.

$$
f(\textbf{e}) = \begin{cases} \textrm{1, si} \sum_{i=1}^{n} {w_i  e_i} \geq \theta \\ \\ 0, \textrm{en caso contrario} \end{cases}
$$

Supongamos que tenemos el siguiente conjunto de muestras, donde los puntos azules corresponden a la clase 1 y los rojos a la clase 0:

<img src="imgs/dosclases01.png" width="50%">

Vemos claramente que no es separable linealmente, pero podríamos hacer que una parte de los datos fueran separados por una neurona y otra parte por otra, como muestra la figura. Las flechas indican la región de activación de cada neurona. Es decir, cuando devuelve un 1.

<img src="imgs/dosclases02.png" width="50%">

Si hacemos los cálculos tenemos que las dos rectas son, para la neurona 1:

$$2x + 3y = 26$$

y para la neurona 2:

$$x - y = 3$$

Si las representáramos gráficamente:

<img src="imgs/dos_neuronas.svg" width="30%">

Si nos fijamos, los elementos de la clase 1 son aquellos donde, simultáneamente, tanto la neurona 1 como la 2 se activan. 

<img src="imgs/dosclases03.png" width="50%">

¿Cómo podríamos combinar las salidas de las dos neuronas? Fácil, haciendo que ambas salidas entren a una tercera. 

<img src="imgs/tresneuronas.svg" width="60%">

Veamos cómo funciona esta red. Supongamos que entra el punto $(5,6)$, esto hace que las neuronas 1 y 2 se activen (generen un 1). Estos unos son multiplicados por sendos unos que son los pesos de la neurona 3. Ambos se suman y el resultado es 2. Este 2 se compara con el 1.5 del umbral de la neurona 3, y, al ser mayor, la neurona 3 también se activa, genera un 1 a la salida. Veamos qué pasa con el punto $(5,3)$. Este punto activa la neurona 2, pero no la 1. Por tanto la suma pesada en la neurona 3 es 1. Esto no supera el umbral de 1.5 y, por tanto, la neurona 3 no se activa, su salida es un 0. Fíjate que la neurona 3 actúa como un *AND* lógico. Hemos hecho nuestra primera red neuronal. Al conjunto de neuronas que reciben como entradas las salidas de otras se le denomina **capa**. Llamaremos **capa de entrada** al conjunto de las entradas iniciales, y **capa de salida** a la neurona o neuronas que generan la salida definitiva. A las capas intermedias se las denomina **capas ocultas**.

-----------------------

## Ejercicio

Calcula los pesos y umbrales de las tres neuronas que delimitan la zona de activación (sombreado azul). Combina las tres neuronas mediante una cuarta que se active solo si las tres salidas de las neuronas anteriores son 1.

<img src="imgs/ejercicio1.png" width="40%">

-----------------------


Hagamos la cosa un poco más complicada (y más interesante). En el siguiente ejemplo vemos que tenemos dos regiones inconexas. Un punto de la clase 1 (azul) puede estar ahora en dos regiones distintas. Ya sabemos cómo hacer una red que se active en la región triangular y otra que se active en la región trapezoidal, pero... ¿cómo combinamos ambas?

<img src="imgs/regiones.png" width="60%">

Ahora un punto de la clase 1 puede estar en una de las dos regiones (obviamente, no en las dos a la vez). La solución es hacer dos redes, una que cubra la región triangular y otra la trapezoidal. Si el punto a tratar está en alguna de estas regiones sombreadas, su correspondiente red generará un 1 (se activará), mientras que la otra generará un 0, y viceversa. De la misma forma que antes introdujimos una neurona que actuaba como *AND* lógico, vamos a introducir otra que actúe como *OR*. Esta nueva neurona conformará una nueva capa. Le pondremos todos sus pesos a 1, ¿y su umbral? Si de todas las regiones que combina (dos en este caso) se activa una de ellas, el valor pesado y sumado de todas sus entradas será, como máximo, 1. Por tanto, el valor del umbral será, por ejemplo, 0.5 (un número mayor que 0 pero menor que 1).


<img src="imgs/redneuronal.svg" width="60%">

-----------------------

## Ejercicio

Construye una red neuronal cuya zona de activación corresponda al área sombreada de la figura. Señala con una flecha el subespacio de activación de cada neurona que utilices en la red, como en la recta a. [Ejercicio de examen 2018] 

<img src="imgs/examen.png" width="30%">

[Resolución del ejercicio](https://www.youtube.com/watch?v=oSOIIpO8PtY&feature=youtu.be)

-----------------------

## Aprendizaje

De la misma forma que ocurría con la neurona individual, la red también puede aprender los pesos de sus conexiones de forma autónoma. Para ello, también es necesario contar con un conjunto de datos etiquetados. En cuanto al interconexionado de la red, hemos visto en los ejemplos anteriores que colocábamos y conectábamos las diferentes neuronas en función de cuántas y cómo eran las regiones que queríamos cubrir. Desafortunadamente, esto no es posible en los casos reales. Para comenzar nuestro estudio utilizamos ejemplos en dos dimensiones para poder visualizar el problema, pero nos podemos encontrar con problemas con mil dimensiones o más, así que la visualización es imposible, y, por tanto, la configuración manual, también. Por ello, la interconexión de las redes se realiza de forma completa. Es decir, las salidas de una capa se conectan a todas las entradas de la siguiente capa. Se deja al proceso de aprendizaje de la red que establezca los pesos adecuadamente.

<img src="imgs/fc.svg" width="60%">

En cuanto al proceso de aprendizaje, es muy similar al explicado para una sola neurona. La red es inicializada con un conjunto de pesos aleatorios. Un *dataset* etiquetado es suministrado a la red para que genere unos resultados que son comparados con sus correspondientes etiquetas mediante una función de error. Mediante la técnica de descenso por el gradiente se calcula un nuevo conjunto de pesos que reduce el error cometido. Se repite el proceso hasta que se alcanza un valor cercano al mínimo (probablemente, un mínimo local de la función). 

-----------------------

## Ejercicio

¿Cuántos pesos distintos y umbrales tendrá una red formada por una capa de entrada de 4 entradas (esto es el **vector de características** con 4 propiedades), una capa oculta de 5 neuronas, otra capa oculta de 4 neuronas y una capa de salida de 1 neurona?

-----------------------


## Dudas

- ¿Cuál es el motivo por el que el valor del umbral de las neuronas es siempre 0'5 menos que el valor máximo? Comprendo que siempre debe ser menor que el resultado esperado, pero no ser mucho menor pues no funcionaría de manera correcta, por ejemplo aquí (http://cayetanoguerra.github.io/ia/nbpy/imgs/tresneuronas.svg) el valor debe cumplir (1<x) y (x<2) por lo que comprendo que 1'1 también sería un valor correcto, al igual que 1'9, entonces, ¿usa usted 1'5 siempre sólo por consistencia? ¿es por tener algún margen de error?

<span style="color:#3660a5">En primer lugar hay que distinguir si la neurona esta haciendo la función de un *AND* o de un *OR*. Supongamos que queremos que actúe como un *AND*. En ese caso, para que la neurona se active, debe cumplirse que **todas sus entradas estén a uno** y, por lo tanto, el umbral debe ser un poco menor que la suma de todas sus entradas pero mayor que la suma de todas sus entradas menos uno. Por ejemplo, si la neurona tiene cuatro entradas, el umbral debe estar comprendido entre un valor algo mayor que $3$ y algo menor que $4$. Un valor posible es $3.5$. Si somos estrictos también nos valdría el valor $4$ ya que la condición de la función de activación es mayor o igual (no solo mayor).</span>

<span style="color:#3660a5">En el caso de un *OR* nos basta con que **al menos una de las entradas esté a uno**. Por tanto, el umbral estará comprendido entre un valor algo mayor que $0$ y menor que $1$.
</span>

<span style="color:#3660a5">En definitiva, en respuesta a tu pregunta, uso $1.5$ (si ese es el caso) porque es más didáctico y estético, nada más.</span>

-----------------------------------

- Tras haber leído todos los apuntes sólo me ha surgido la siguiente duda. Las muestras que caen sobre la recta de la neurona, ¿siempre se clasifican en la misma región? o puede pasar que distintos puntos sobre la misma recta se clasifiquen en regiones diferentes.

<span style="color:#3660a5">Cuando hacemos uso de una función de activación tipo "mayor o igual", todos los puntos sobre la recta caen en el lado "mayor o igual", ya que su coordenada $y$ es igual a la coordenada de la recta. Por tanto, todos los puntos sobre la recta caen en el lado de la activación.</span>
